In [237]:
import os
import re
import pandas as pd 

class TF_Data_Processing():
    
    def __init__(self,dir_path,cancer,select_slice_path,save_path):

        self.cancer=cancer
        self.dir_path=dir_path
        self.save_path=save_path
        self.select_slice_path=select_slice_path
        self.file_name=os.listdir(self.dir_path)
        self.file_path= [os.path.join(self.dir_path,i) for i in os.listdir(self.dir_path)]
    


    def Create_folder(self):

        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)



    def Single_slice_data_extraction(self):

        all_slice=pd.read_csv(self.select_slice_path)["slice"].values
        for i in self.file_name:
            cancer=re.findall(r'\D+', i.split("-")[0])
            slice=i.split(".loom.tsv")[0]
            if cancer[0] == self.cancer and slice in all_slice:
                data=pd.read_csv(os.path.join(self.dir_path,i),sep="\t")
                data["TF"].value_counts().to_csv(self.save_path+"\\"+slice+".csv",columns=["TF"])
    


    def TF_slice_num_statistics(self):

        TF_file_path= [os.path.join(self.save_path,i) for i in os.listdir(self.save_path)]
        dfs = []
        for file in TF_file_path:
            cancer = re.findall(self.cancer, file)
            if cancer[0]==self.cancer:
                df = pd.read_table(file,sep=",",index_col=0)
                dfs.append(df)
        combined_df = pd.concat(dfs,axis=0).index.value_counts()
        combined_df=pd.DataFrame(combined_df)
        combined_df.columns=["slice_num"]
        combined_df["all_slice_num"]=[len(TF_file_path) for i in range(len(combined_df.index))]
        combined_df["ratio"]=(combined_df["slice_num"]/combined_df["all_slice_num"]).round(3)
        combined_df.to_csv(os.path.join(self.save_path,"TF-slice-num.txt"),sep="\t")
        print(f"{self.cancer}的TF-slice数目统计完成了!")
        print("******************************************************************************************************************")
        print(combined_df)



    def Drop_single_slice_file(self):

        files=os.listdir(self.save_path)
        for file in files:
            cancer=re.findall(r'\D+', file.split("-")[0])
            if cancer[0]==self.cancer:
                file_path = os.path.join(self.save_path, file)
                os.remove(file_path)
    



    def TF_terget_statistics(self):

        dfs = []
        select_slice=pd.read_csv(self.select_slice_path)["slice"].values
        for i in self.file_name:
            cancer=re.findall(r'\D+', i.split("-")[0])
            slice=i.split(".loom.tsv")[0]
            if cancer[0] == self.cancer and slice in select_slice:
                data=pd.read_csv(os.path.join(self.dir_path,i),sep="\t")
                data["slice"]=[slice for i in range(len(data.index))]
                data["cancer"]=[cancer[0] for i in range(len(data.index))]
                dfs.append(data)
        combined_df = pd.concat(dfs)
        group_list=combined_df.groupby(["TF","slice"])
        num=0
        table=pd.DataFrame(columns=["TF","target","target_num","importance","slice","cancer"])
        for group_name, group_data in group_list:
            target=[i for i in group_data["target"]]
            importance=[str(i) for i in group_data["importance"].round(3)]
    
            cancer=[i for i in group_data["cancer"]]
            target_num=len(target)
            table.loc[num,"TF"]=group_name[0]
            table.loc[num,"target"]=",".join(target)
            table.loc[num,"target_num"]=target_num
            table.loc[num,"importance"]=",".join(importance)
            table.loc[num,"slice"]=group_name[1].replace("-","_")
            table.loc[num,"cancer"]=self.cancer
            num+=1
        table.to_csv(self.save_path+"\\TF-Statistics"+".txt",sep="\t",index=False)
        print(f"{self.cancer}的TF-target统计完成了!")
        print("******************************************************************************************************************")
        print(table)
        
                

In [238]:
cancer=["hn_as"]
for i in cancer:
    print(f"开始{i}的TF的数据统计")
    test=TF_Data_Processing(dir_path="result_grn",cancer=i,select_slice_path="E:\苏师兄代码和文件\select_slice1.csv",save_path=i.upper())
    test.Create_folder()
    test.Single_slice_data_extraction()
    test.TF_slice_num_statistics()
    test.Drop_single_slice_file()
    test.TF_terget_statistics()
print("所有统计完成")

开始hn_as的TF的数据统计


IndexError: list index out of range

In [ ]:
#总表统计
import os
import pandas as pd
dfs=[]
for i in os.listdir("E:\苏师兄代码和文件\pyscenic"):
    if os.path.exists(os.path.join(r"E:\苏师兄代码和文件\pyscenic",i+"\\TF-slice-num.txt")):
        data=pd.read_csv(os.path.join(r"E:\苏师兄代码和文件\pyscenic",i+"\\TF-slice-num.txt"),sep="\t")
        dfs.append(data)
combined_df = pd.concat(dfs)
combined_df = combined_df.rename(columns={combined_df.columns[0]: "TF"})
table=pd.DataFrame()
table["TF"]=combined_df.groupby("TF")["slice_num"].sum().index
table["slice_num"]=combined_df.groupby("TF")["slice_num"].sum().values
table["all_slice_num"]=[144 for i in range(len(table["TF"]))]
table["slice_ratio"]=(table["slice_num"]/table["all_slice_num"]).round(3)
table["cancer_num"]=combined_df["TF"].value_counts().values
table["all_cancer_num"]=[18 for i in range(len(table["TF"]))]
table["cancer_ratio"]=(table["cancer_num"]/table["all_cancer_num"]).round(3)
table.to_csv("ALL\\All-TF-slice-num.txt",sep="\t",index=False)

In [ ]:
table

,TF,slice_num,all_slice_num,slice_ratio,cancer_num,all_cancer_num,cancer_ratio
0,A1CF,45,144,0.312,18,18,1.000
1,ABCF2,114,144,0.792,18,18,1.000
2,ABL1,143,144,0.993,18,18,1.000
3,ACAA1,143,144,0.993,18,18,1.000
4,ACO1,143,144,0.993,18,18,1.000
...,...,...,...,...,...,...,...
1815,ZSWIM1,139,144,0.965,2,18,0.111
1816,ZXDA,106,144,0.736,2,18,0.111
1817,ZXDB,138,144,0.958,2,18,0.111
1818,ZXDC,143,144,0.993,1,18,0.056


In [ ]:
#总表统计
import os
import pandas as pd
dfs=[]
for i in os.listdir("E:\苏师兄代码和文件\pyscenic"):
    if os.path.exists(os.path.join(r"E:\苏师兄代码和文件\pyscenic",i+"\\TF-Statistics.txt")):
        data=pd.read_csv(os.path.join(r"E:\苏师兄代码和文件\pyscenic",i+"\\TF-Statistics.txt"),sep="\t")
        dfs.append(data)
combined_df = pd.concat(dfs)
combined_df = combined_df.rename(columns={combined_df.columns[0]: "TF"})
combined_df=combined_df.sort_values(by='TF', ascending=True)
combined_df.to_csv("ALL\\All-TF-Statistics.txt",sep="\t",index=False)

In [ ]:
combined_df["cancer"]

,TF,target,target_num,importance,slice,cancer
10,A1CF,"DRC3,YOD1,AC147651.1,CHRNA4,TAB1,FAM242C,ANKFY...",370,"1.8,1.472,1.384,1.374,1.093,1.09,1.066,0.945,0...",crc10_slice3,crc
14,A1CF,"SLC4A9,LINC02577,CHAC1,AL365181.4,AC004241.3,Z...",3035,"5.043,4.324,3.806,3.623,3.399,2.943,2.917,2.87...",lihc03_slice2,lihc
13,A1CF,"LINC01339,AC004593.2,ZNF607,LINC02518,AL355297...",2241,"5.412,4.368,3.977,3.608,3.297,2.84,2.804,2.482...",lihc03_slice1,lihc
12,A1CF,"VPREB3,C1orf52,SNCB,AL354919.1,ZNF417,LTBP1,US...",2671,"2.59,2.509,2.397,2.165,2.103,1.984,1.969,1.959...",lihc02_slice9,lihc
1,A1CF,"TMPRSS15,PLA2G6,MPP7,ZBTB47,SETBP1,TNXB,TRBJ1-...",223,"1.174,1.103,0.972,0.811,0.698,0.557,0.552,0.46...",pdac03_slice2,pdac
...,...,...,...,...,...,...
52172,ZXDC,"RAB33B,DUSP9,IPO8,POLH,HSF4,SELPLG,CCNC,SNHG18...",360,"2.204,2.061,1.939,1.894,1.88,1.815,1.647,1.646...",gbm05_slice14,gbm
52163,ZXDC,"DNAAF3,AC010931.3,CPLX3,LCE1D,PARD6G,ZNF264,AL...",1004,"3.096,2.187,1.728,1.382,1.245,1.112,1.104,1.03...",gbm04_slice4,gbm
2849,ZXDC,"CRISP1,TULP2,PLA2G2C,LDOC1,SPSB4,KRTAP26-1,ZNF...",783,"2.611,1.238,1.004,1.003,0.927,0.914,0.86,0.846...",skcm13_slice1,skcm
2479,ZZZ3,"SLC6A1,MAFTRR,PDS5B,LOC103344931,ST6GALNAC3,AR...",748,"5.038,3.731,3.717,3.077,2.848,2.224,2.072,1.74...",rcc01_slice2,rcc


In [ ]:
combined_df["TF"].value_counts()

MBD1       143
CBFA2T2    143
ZNF316     143
DNMT3A     143
ZNF397     143
          ... 
ZNF735       3
FOXD4L4      2
TGIF2LY      2
ZZZ3         2
SSX2         1
Name: TF, Length: 1820, dtype: int64

In [ ]:
import os
import re
import  ast
import pandas as pd 

In [ ]:
data=pd.read_csv(r"E:\苏师兄代码和文件\pyscenic\result_ctx\reg_brca01-slice1.loom.tsv.csv")
data.columns=data.iloc[0,:]
data=data[1:]
data.columns.values[0]="TF"
data.columns.values[1]="MotifID"
data=data[1:]


In [ ]:
Target_num=[]
for i in data["TargetGenes"]:
    pattern = re.compile('[a-zA-Z]+')
    english_words = [re.findall(pattern, item[0])[0] for item in ast.literal_eval(i)]
    Target_num.append(len(english_words))
data["Target_num"]=Target_num


In [ ]:
data=data.sort_values(by="Target_num",ascending=False)
data

,TF,MotifID,AUC,NES,MotifSimilarityQvalue,OrthologousIdentity,Annotation,Context,TargetGenes,RankAtMax,Target_num
101,XBP1,metacluster_71.16,0.04442817365320294,3.186038904705333,9.44332e-08,1.0,motif similar to taipale_cyt_meth__XBP1_NKMCAC...,"frozenset({'activating', 'weight>75.0%', 'hg38...","[('CNOT2', 0.4836674489060573), ('PEX2', 1.661...",4970,529
675,ELF3,metacluster_191.2,0.04968564357473354,3.1943287350642726,6.90333e-06,1.0,motif similar to taipale_cyt_meth__ELF3_ACCCGG...,"frozenset({'activating', 'top50perTarget', 'hg...","[('SOCS6', 9.443884364310829e-08), ('SERTAD2',...",4905,316
742,STAT1,tfdimers__MD00523,0.0435436743791873,3.182879452680371,1.91863e-09,1.0,gene is annotated for similar motif tfdimers__...,"frozenset({'activating', 'top50perTarget', 'hg...","[('FBLN7', 0.4067316237631928), ('SHC2', 0.100...",4955,316
738,SPDEF,taipale_tf_pairs__GCM1_ELK1_RTGCGGGCGGAAGTN_CAP_1,0.044129575396384936,3.0311327308119522,3.95882e-08,1.0,motif similar to taipale_tf_pairs__GCM1_SPDEF_...,"frozenset({'activating', 'top50perTarget', 'hg...","[('ACSL4', 0.085369573176543), ('SLC45A4', 0.4...",4737,298
737,SPDEF,metacluster_166.2,0.04830189860628467,3.793683640426003,1.83208e-07,1.0,motif similar to taipale_tf_pairs__ELK1_SPDEF_...,"frozenset({'activating', 'top50perTarget', 'hg...","[('NPRL3', 1.1261173811912362), ('C1GALT1', 0....",4973,294
...,...,...,...,...,...,...,...,...,...,...,...
498,PPARG,tfdimers__MD00172,0.11869592741084617,3.0763678507109002,5.77514e-08,1.0,gene is annotated for similar motif tfdimers__...,"frozenset({'top5perTarget', 'activating', 'hg3...","[('SORBS3', 0.9420932782059972), ('DEDD', 0.59...",88,2
416,CLOCK,taipale_tf_pairs__CLOCK_BHLHA15_NCACGTGNNNNNCA...,0.155480059084195,4.447664461124356,0.0,1.0,gene is directly annotated,"frozenset({'top5perTarget', 'activating', 'hg3...","[('NUP35', 0.7917121613195244), ('CAMK2N1', 1....",53,2
186,HOXC4,metacluster_9.9,0.10955194485475135,3.198181842922439,6.45702e-06,1.0,motif similar to swissregulon__hs__HOXC4 ('hs_...,"frozenset({'activating', 'hg38_10kbp_up_10kbp_...","[('KDSR', 0.9848095898039196), ('HOXC4', 1.0)]",71,2
406,ATF3,tfdimers__MD00565,0.14307401936648614,3.4502281215356465,1.54335e-06,1.0,motif similar to tfdimers__MD00119 ('M00973_re...,"frozenset({'top5perTarget', 'activating', 'hg3...","[('EFNA3', 1.0427155236199108), ('ZNF292', 0.7...",51,2


In [261]:
cancerlist=["brca","cesc","crc","cscc","gbm","gist","hn_as","ipnm","lihc","luad","mibc","oscc","ovca","pcnsl","pdac","prad","rcc","skcm"][0]
dir_path="result_ctx"
select_slice_path="E:\苏师兄代码和文件\select_slice1.csv"
file_name=os.listdir(dir_path)
file_path= [os.path.join(dir_path,i) for i in os.listdir(dir_path)]
dfs = []
select_slice=pd.read_csv(r"E:\苏师兄代码和文件\select_slice1.csv")["slice"].values
for i in file_name:
    cancer=re.findall(r'\D+', i.split("-")[0])[0].replace("reg_","")
    slice=i.split(".loom.tsv.csv")[0].replace("reg_","")
    if cancer == "brca" and slice in select_slice:
        data=pd.read_csv(r"E:\\苏师兄代码和文件\\pyscenic\\result_ctx\\"+i)
        data.columns=data.iloc[0,:]
        data=data[1:]
        data.columns.values[0]="TF"
        data.columns.values[1]="MotifID"
        data=data[1:]
        data["slice"]=[slice for i in range(len(data.index))]
        data["cancer"]=[cancer for i in range(len(data.index))]
        dfs.append(data)
combined_df = pd.concat(dfs)

In [262]:
combined_df

,TF,MotifID,AUC,NES,MotifSimilarityQvalue,OrthologousIdentity,Annotation,Context,TargetGenes,RankAtMax,slice,cancer
2,ATF4,metacluster_148.1,0.04535987163002242,3.094187231280473,0.0,1.0,gene is directly annotated,frozenset({'hg38_10kbp_up_10kbp_down_full_tx_v...,"[('CYHR1', 0.7961399861048666), ('ATF3', 0.549...",4316,brca01-slice3,brca
3,ATF7,tfdimers__MD00403,0.07734985769355478,3.3327772737947217,2.25962e-06,1.0,motif similar to tfdimers__MD00119 ('M00973_re...,frozenset({'hg38_10kbp_up_10kbp_down_full_tx_v...,"[('TBL1XR1', 0.3794181419999426), ('TP73', 0.9...",2860,brca01-slice3,brca
4,BRF1,metacluster_172.30,0.07246964091972653,3.393020859843327,0.0,1.0,gene is directly annotated,frozenset({'hg38_10kbp_up_10kbp_down_full_tx_v...,"[('APEH', 0.4085328157090424), ('KIAA0930', 0....",3481,brca01-slice3,brca
5,CEBPD,metacluster_188.6,0.08619285687232511,3.0759202055964154,4.11114e-06,1.0,gene is annotated for similar motif tfdimers__...,frozenset({'hg38_10kbp_up_10kbp_down_full_tx_v...,"[('PPM1H', 2.3367527585852383), ('WRAP73', 0.4...",920,brca01-slice3,brca
6,CEBPD,metacluster_127.2,0.08783092830360187,3.174015480466356,4.30447e-08,1.0,gene is annotated for similar motif tfdimers__...,frozenset({'hg38_10kbp_up_10kbp_down_full_tx_v...,"[('WRAP73', 0.4242448518158827), ('HERPUD2', 0...",573,brca01-slice3,brca
...,...,...,...,...,...,...,...,...,...,...,...,...
1247,ZNF704,metacluster_144.3,0.052477081428336965,3.451496379493853,0.0,1.0,gene is directly annotated,"frozenset({'activating', 'top50perTarget', 'hg...","[('ZNF704', 1.0), ('SIAH1', 0.2095099932575139...",4645,brca26-slice1,brca
1248,ZNF707,jaspar__MA1715.1,0.06133502145318984,3.2644445065952743,0.0,1.0,gene is directly annotated,"frozenset({'activating', 'top50perTarget', 'hg...","[('PIK3R1', 0.2066647993658202), ('ITSN2', 0.1...",3020,brca26-slice1,brca
1249,ZNF846,metacluster_133.2,0.06818181818181818,3.193544830125133,2.55708e-06,1.0,motif similar to kznf__ZNF846_Imbeault2017_RP_...,"frozenset({'activating', 'top50perTarget', 'hg...","[('CLSTN1', 0.3020024914065943), ('CCDC174', 0...",356,brca26-slice1,brca
1250,ZNF846,kznf__ZNF846_Imbeault2017_OM_MEME,0.0662720260209182,3.025576739632648,0.0,1.0,gene is directly annotated,"frozenset({'activating', 'top50perTarget', 'hg...","[('SECTM1', 0.4894350004497896), ('CDKL5', 0.5...",381,brca26-slice1,brca


In [314]:
cancerlist=["brca","cesc","crc","cscc","gbm","gist","hn_as","ipnm","lihc","luad","mibc","oscc","ovca","pcnsl","pdac","prad","rcc","skcm"][0]
dir_path="result_ctx"
select_slice_path="E:\苏师兄代码和文件\select_slice1.csv"
file_name=os.listdir(dir_path)
file_path= [os.path.join(dir_path,i) for i in os.listdir(dir_path)]
dfs = []
select_slice=pd.read_csv(r"E:\苏师兄代码和文件\select_slice1.csv")["slice"].values
for i in file_name:
    cancer=re.findall(r'\D+', i.split("-")[0])[0].replace("reg_","")
    slice=i.split(".loom.tsv.csv")[0].replace("reg_","")
    if cancer == "brca" and slice in select_slice:
        data=pd.read_csv(r"E:\\苏师兄代码和文件\\pyscenic\\result_ctx\\"+i)
        data.columns=data.iloc[0,:]
        data=data[1:]
        data.columns.values[0]="TF"
        data.columns.values[1]="MotifID"
        data=data[1:]
        data["slice"]=[slice for i in range(len(data.index))]
        data["cancer"]=[cancer for i in range(len(data.index))]
        dfs.append(data)
combined_df = pd.concat(dfs)
table=pd.DataFrame(columns=["TF","target","target_num","slice","cancer","NES"])
table["TF"]=combined_df["TF"]
table["slice"]=combined_df["slice"]
table["cancer"]=combined_df["cancer"]
table["NES"]=combined_df["NES"]
num=0
Target_num=[]
Target=[]
for i,Str in enumerate(combined_df["TargetGenes"]):
    pattern = re.compile('[a-zA-Z]+')
    target = [re.findall(pattern, item[0])[0] for item in eval(Str)]
    Target_num.append(len(target))
    Target.append(target)
    table.iloc[i,1]=",".join(target)
    num=num+1
table["target_num"]=Target_num
table1=table.sort_values(by="target_num",ascending=False)[:10]
table1=table1.sort_values(by="NES",ascending=False)[:10]
table1.to_csv("Top10_sort_by_NES.csv",index=False)

In [313]:
cancerlist=["brca","cesc","crc","cscc","gbm","gist","hn_as","ipnm","lihc","luad","mibc","oscc","ovca","pcnsl","pdac","prad","rcc","skcm"][0]
dir_path="result_ctx"
select_slice_path="E:\苏师兄代码和文件\select_slice1.csv"
file_name=os.listdir(dir_path)
file_path= [os.path.join(dir_path,i) for i in os.listdir(dir_path)]
dfs = []
select_slice=pd.read_csv(r"E:\苏师兄代码和文件\select_slice1.csv")["slice"].values
for i in file_name:
    cancer=re.findall(r'\D+', i.split("-")[0])[0].replace("reg_","")
    slice=i.split(".loom.tsv.csv")[0].replace("reg_","")
    if cancer == "brca" and slice in select_slice:
        data=pd.read_csv(r"E:\\苏师兄代码和文件\\pyscenic\\result_ctx\\"+i)
        data.columns=data.iloc[0,:]
        data=data[1:]
        data.columns.values[0]="TF"
        data.columns.values[1]="MotifID"
        data=data[1:]
        data["slice"]=[slice for i in range(len(data.index))]
        data["cancer"]=[cancer for i in range(len(data.index))]
        dfs.append(data)
combined_df = pd.concat(dfs)
table=pd.DataFrame(columns=["TF","target","target_num","slice","cancer","AUC"])
table["TF"]=combined_df["TF"]
table["slice"]=combined_df["slice"]
table["cancer"]=combined_df["cancer"]
table["AUC"]=combined_df["AUC"]
num=0
Target_num=[]
Target=[]
for i,Str in enumerate(combined_df["TargetGenes"]):
    pattern = re.compile('[a-zA-Z]+')
    target = [re.findall(pattern, item[0])[0] for item in eval(Str)]
    Target_num.append(len(target))
    Target.append(target)
    table.iloc[i,1]=",".join(target)
    num=num+1
table["target_num"]=Target_num
table1=table.sort_values(by="target_num",ascending=False)[:10]
table1=table1.sort_values(by="AUC",ascending=False)[:10]
table1.to_csv("Top10_sort_by_AUC.csv",index=False)

In [ ]:
# combined_df["TargetGenes_name"]=english_words
num=0


for group_name, group_data in group_list:
    target=[i for i in group_data["target"]]
    importance=[str(i) for i in group_data["importance"].round(3)]

    cancer=[i for i in group_data["cancer"]]
    target_num=len(target)
    table.loc[num,"TF"]=group_name[0]
    table.loc[num,"target"]=",".join(target)
    table.loc[num,"target_num"]=target_num
    table.loc[num,"importance"]=",".join(importance)
    table.loc[num,"slice"]=group_name[1].replace("-","_")
    table.loc[num,"cancer"]=self.cancer
    num+=1
table.to_csv(self.save_path+"\\TF-Statistics"+".txt",sep="\t",index=False)
print(f"{self.cancer}的TF-target统计完成了!")
print("******************************************************************************************************************")
print(table)
        

,TF,MotifID,AUC,NES,MotifSimilarityQvalue,OrthologousIdentity,Annotation,Context,TargetGenes,RankAtMax,slice,cancer,Target_num
2,ATF4,metacluster_148.1,0.04535987163002242,3.094187231280473,0.0,1.0,gene is directly annotated,frozenset({'hg38_10kbp_up_10kbp_down_full_tx_v...,"[('CYHR1', 0.7961399861048666), ('ATF3', 0.549...",4316,brca01-slice3,b,106
3,ATF7,tfdimers__MD00403,0.07734985769355478,3.3327772737947217,2.25962e-06,1.0,motif similar to tfdimers__MD00119 ('M00973_re...,frozenset({'hg38_10kbp_up_10kbp_down_full_tx_v...,"[('TBL1XR1', 0.3794181419999426), ('TP73', 0.9...",2860,brca01-slice3,b,31
4,BRF1,metacluster_172.30,0.07246964091972653,3.393020859843327,0.0,1.0,gene is directly annotated,frozenset({'hg38_10kbp_up_10kbp_down_full_tx_v...,"[('APEH', 0.4085328157090424), ('KIAA0930', 0....",3481,brca01-slice3,b,32
5,CEBPD,metacluster_188.6,0.08619285687232511,3.0759202055964154,4.11114e-06,1.0,gene is annotated for similar motif tfdimers__...,frozenset({'hg38_10kbp_up_10kbp_down_full_tx_v...,"[('PPM1H', 2.3367527585852383), ('WRAP73', 0.4...",920,brca01-slice3,b,11
6,CEBPD,metacluster_127.2,0.08783092830360187,3.174015480466356,4.30447e-08,1.0,gene is annotated for similar motif tfdimers__...,frozenset({'hg38_10kbp_up_10kbp_down_full_tx_v...,"[('WRAP73', 0.4242448518158827), ('HERPUD2', 0...",573,brca01-slice3,b,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247,ZNF704,metacluster_144.3,0.052477081428336965,3.451496379493853,0.0,1.0,gene is directly annotated,"frozenset({'activating', 'top50perTarget', 'hg...","[('ZNF704', 1.0), ('SIAH1', 0.2095099932575139...",4645,brca26-slice1,b,186
1248,ZNF707,jaspar__MA1715.1,0.06133502145318984,3.2644445065952743,0.0,1.0,gene is directly annotated,"frozenset({'activating', 'top50perTarget', 'hg...","[('PIK3R1', 0.2066647993658202), ('ITSN2', 0.1...",3020,brca26-slice1,b,32
1249,ZNF846,metacluster_133.2,0.06818181818181818,3.193544830125133,2.55708e-06,1.0,motif similar to kznf__ZNF846_Imbeault2017_RP_...,"frozenset({'activating', 'top50perTarget', 'hg...","[('CLSTN1', 0.3020024914065943), ('CCDC174', 0...",356,brca26-slice1,b,13
1250,ZNF846,kznf__ZNF846_Imbeault2017_OM_MEME,0.0662720260209182,3.025576739632648,0.0,1.0,gene is directly annotated,"frozenset({'activating', 'top50perTarget', 'hg...","[('SECTM1', 0.4894350004497896), ('CDKL5', 0.5...",381,brca26-slice1,b,12
